In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    if pd.isnull(text):
        return ''
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return ' '.join(filtered_tokens)

def load_data(category_url, chunk_size, total_samples):
    chunks = pd.read_json(category_url, lines=True, chunksize=chunk_size)
    data = pd.concat([chunk for chunk in chunks], ignore_index=True)
    data = data.sample(min(total_samples, len(data)))
    return data

def preprocess_data(data):
    data = data.dropna(subset=['reviewText'])
    data['reviewText'] = data['reviewText'].apply(preprocess_text)
    return data


In [3]:
def train_decision_tree(X_train_tfidf, y_train, param_grid):
    dt_model = DecisionTreeClassifier()
    grid_search = GridSearchCV(estimator=dt_model, param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train_tfidf, y_train)
    best_params = grid_search.best_params_
    best_model = DecisionTreeClassifier(**best_params)
    best_model.fit(X_train_tfidf, y_train)
    return best_model

def evaluate_model(model, X, y, set_name):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    print(f'{set_name} Accuracy: {accuracy:.2f}')
    print(f'Classification Report ({set_name}):')
    print(classification_report(y, y_pred))
    


In [7]:
def main():
    categories = {
        'Luxury_Beauty': 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/Luxury_Beauty.json.gz',
        'AMAZON_FASHION': 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/AMAZON_FASHION.json.gz',
        'All_Beauty': 'https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_v2/categoryFiles/All_Beauty.json.gz'
    }
    total_samples = 10000
    chunk_size = 10000
    
    param_grid = {
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Load and preprocess training data for AMAZON_FASHION and All_Beauty
    train_data_amazon_fashion = preprocess_data(load_data(categories['AMAZON_FASHION'], chunk_size, total_samples))
    train_data_all_beauty = preprocess_data(load_data(categories['All_Beauty'], chunk_size, total_samples))
    
    # Combine training data
    train_data = pd.concat([train_data_amazon_fashion, train_data_all_beauty], ignore_index=True)

    # Separate features (reviews) and target variable (ratings)
    X_train = train_data['reviewText']
    y_train = train_data['overall']

    # Convert text data to TF-IDF features
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    X_train_tfidf = vectorizer.fit_transform(X_train)

    # Split training data into training and validation sets
    X_train_split, X_validation_split, y_train_split, y_validation_split = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

    # Train Decision Tree model
    best_model = train_decision_tree(X_train_split, y_train_split, param_grid)

    # Print information about the best parameters
    print(f'Best Parameters: {best_model.get_params()}')

    # Evaluate the model on the validation set
    evaluate_model(best_model, X_validation_split, y_validation_split, 'Validation Set')

    # Load and preprocess testing data for Luxury_Beauty
    test_data = preprocess_data(load_data(categories['Luxury_Beauty'], chunk_size, total_samples))

    # Separate features and target variable for testing data
    X_test = test_data['reviewText']
    y_test = test_data['overall']
    X_test_tfidf = vectorizer.transform(X_test)

    # Evaluate the model on the test set
    evaluate_model(best_model, X_test_tfidf, y_test, 'Test Set')

if __name__ == "__main__":
    main()


/var/folders/55/kz51hsm17hx84shxkkwr1y7r0000gn/T/ipykernel_29692/2205971309.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['reviewText'] = data['reviewText'].apply(preprocess_text)
/var/folders/55/kz51hsm17hx84shxkkwr1y7r0000gn/T/ipykernel_29692/2205971309.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['reviewText'] = data['reviewText'].apply(preprocess_text)


Best Parameters: {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 5, 'min_weight_fraction_leaf': 0.0, 'random_state': None, 'splitter': 'best'}
Validation Set Accuracy: 0.60
Classification Report (Validation Set):
              precision    recall  f1-score   support

           1       0.43      0.28      0.34       450
           2       0.29      0.02      0.03       251
           3       0.13      0.01      0.01       402
           4       0.22      0.02      0.03       571
           5       0.63      0.97      0.76      2323

    accuracy                           0.60      3997
   macro avg       0.34      0.26      0.24      3997
weighted avg       0.47      0.60      0.49      3997



/var/folders/55/kz51hsm17hx84shxkkwr1y7r0000gn/T/ipykernel_29692/2205971309.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['reviewText'] = data['reviewText'].apply(preprocess_text)


Test Set Accuracy: 0.67
Classification Report (Test Set):
              precision    recall  f1-score   support

           1       0.42      0.14      0.21       884
           2       0.16      0.01      0.01       510
           3       0.20      0.01      0.01       747
           4       0.20      0.01      0.03      1211
           5       0.68      0.98      0.80      6642

    accuracy                           0.67      9994
   macro avg       0.33      0.23      0.21      9994
weighted avg       0.54      0.67      0.56      9994

